In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output
import pickle

#common utilities for transforming data
from commons import Utilities
utils = Utilities()

In [1]:
DATA_DIR = "../input"

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
CHUNKSIZE = 180000

TRAIN_NUMERIC = "{0}/train_num.csv".format(DATA_DIR)
TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)

TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)
TEST_DATE = "{0}/test_date.csv".format(DATA_DIR)


In [ ]:
'''
print("Finding medians of columns in chunks...")
medians=[]

chunknum = 0
for chunk in pd.read_csv(TRAIN_NUMERIC, header=0, chunksize=CHUNKSIZE):
    median_chunk = Utilities.findNa(utils,chunk,chunknum)
    medians.append(median_chunk)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*CHUNKSIZE/1183748)*100))
    
medians = pd.DataFrame(medians)
'''

In [ ]:
#pickle.dump( medians, open( "../exported/medians.p", "wb" ) )

In [ ]:
#medians = pickle.load( open( "../exported/medians.p", "rb" ) )

In [3]:
#Train
models=[]
def train(df):
    print("Filling missing data with median values...")
    #df = utils.normalize_data_w_medians(df,medians)
    df['Id2'] = df[ID_COLUMN] 
    print("Preparing data for training...")
    X = df.drop([ID_COLUMN,TARGET_COLUMN],axis=1)
    y = df[TARGET_COLUMN]
    
    # Random Forests 
    model = RandomForestClassifier(n_estimators=100)
    print("Training...")
    model.fit(X,y)
    models.append(model)

print("Reading train data into memory in chunks...")
chunknum = 0
for chunk in pd.read_csv(TRAIN_NUMERIC, chunksize=CHUNKSIZE):
    train(chunk)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*CHUNKSIZE/1183748)*100))
pickle.dump( models, open( "../exported/models.p", "wb" ) )

106 percent
